## Introduction

In this project, my friend Nirvit and I shared our 2023 Spotify Wrapped playlists so we could visualize comparisons between our music tastes and then create a model to try and predict whose playlist a song belongs to. Finally, I have compiled the results of each model into an interactive dashboard using [Panel](https://panel.holoviz.org/). 

This blog post will have the following sections:

* Setup and Preprocessing

* Exploratory Data Analysis for Feature Selection

* Prepping Data For Machine Learning Models

* Creating Machine Learning Models

* Panel Dashboard

* Final thoughts 

Now, let’s dive into the exciting world of music data analysis!

### Understanding our Spotify Dataset

**Track Metadata**
| column | description |
| --- | --- |
| Song | Song title |
| Artist | Song artist |
| Genre | Song genre category |

**Audio Numerical Quantitive Data**
| column | description |
| --- | --- |
| Loud | How loud a song is (db) |
| Time Seconds | Duration of the song in seconds |
| BPM | Average song tempo / how fast a song is |

**Audio Qualitative Data**
| column | description |
| --- | --- |
| Energy | How energetic the song is |
| Dance | How easy the song is to dance to |
| Happy | How positive the mood of the song is |
| Acoustic | How acoustic sounding the song is |
| Speech | How much of a song is spoken word |
| Popularity |  How popular a song is (at time of data collection) |
| Live | How likely the song is a live recording (higher value = live recording) |
| Instrumental | Measures if the song is more music and less vocals |


**Audio Categorical Data**
| column | description |
| --- | --- |
| Key | The most repeated key in the song |
| Time Signature | Numerical representation of rhythmic structure in song |
| Camelot | Musical key of a song for harmonic mixing |
| Playlist Owner | Who's playlist the song belongs to |


[**Gather Your Own Spotify Dataset**](https://www.chosic.com/spotify-playlist-analyzer/?plid=37i9dQZF1Fa1IIVtEpGUcU)

## Setup and Preprocessing


In [ ]:
import pandas as pd
import numpy as np

# Read in csv file to create tabular dataframe 
piero_top_songs = pd.read_csv("/Users/brizaespinoza/Downloads/Git/suppiero.github.io/projects/spotify_classification_dashboard/data/Piero_Top_Songs_2023.csv") 
nirvit_top_songs = pd.read_csv("/Users/brizaespinoza/Downloads/Git/suppiero.github.io/projects/spotify_classification_dashboard/data/Nirvit_Top_Songs_2023.csv") 

# Add a truth column to classify whether a song is from Piero's or Nirvit's playlist
piero_top_songs['Playlist Owner'] = 'Piero'
nirvit_top_songs['Playlist Owner'] = 'Nirvit'

# Convert time to seconds
piero_top_songs['Time Seconds'] = pd.to_timedelta('00:' + piero_top_songs['Time']).dt.total_seconds().astype(int)
nirvit_top_songs['Time Seconds'] = pd.to_timedelta('00:' + nirvit_top_songs['Time']).dt.total_seconds().astype(int)

# Remove unnecessary columns
piero_top_songs = piero_top_songs.drop(columns=['Song Preview', 'Spotify Track Img', 'Album Label', 'Spotify Track Id', 'Added At', 'Spotify Track Id', '#', 'Album', 'Album Date', 'Time'])
nirvit_top_songs = nirvit_top_songs.drop(columns=['Song Preview', 'Spotify Track Img', 'Album Label', 'Spotify Track Id', 'Added At', 'Spotify Track Id', '#', 'Album', 'Album Date', 'Time'])

# Join playlists into one dataframe
all_songs = pd.concat([piero_top_songs, nirvit_top_songs])

# Convert all object columns to type string
object_columns = all_songs.select_dtypes(include=['object']).columns # First, create list of object columns to convert
all_songs[object_columns] = all_songs[object_columns].astype('string')

#print(all_songs.dtypes) # Check that column types have been converted to string

In [ ]:
# Check for null values
all_songs.isnull().sum().sum() # 9 NaN values in 'Genres' and 'Parent Genres' columns

# Create dataframe of songs containing NaN values in either 'Genres' or 'Parent Genres'
nan_rows = all_songs[(all_songs['Genres'].isnull()) | (all_songs['Parent Genres'].isnull())]

# Fill NaNs in 'Parent Genres' column with 'Unknown' since I cannot find them on Spotify or Google
all_songs[['Genres']] = all_songs[['Genres']].fillna('Unknown')

# Populate NaN values in 'Genres' column with genres found on Spotify or Google for specified song and artist
all_songs.loc[(all_songs['Song'] == 'Mumbo Sugar') & (all_songs['Artist'] == 'Arc De Soleil'), ['Parent Genres']] = ['R&B, Soul']

all_songs.loc[(all_songs['Song'] == 'Give It Back') & (all_songs['Artist'] == 'Gaelle'), ['Parent Genres']] = ['Dance, Electronic']

all_songs.loc[(all_songs['Song'] == '愛してる') & (all_songs['Artist'] == "callin'"), ['Parent Genres']] = ['Anime, J-Pop']

all_songs.loc[(all_songs['Song'] == 'You Are Mine') & (all_songs['Artist'] == 'Jay Robinson'), ['Parent Genres']] = ['Classic Soul']

all_songs.loc[(all_songs['Song'] == 'Thank You DubNation! (the page will never be long enough)') & (all_songs['Artist'] == 'herlovebeheadsdaisies'), ['Parent Genres']] = ['Screamo']

# Convert categorical variables to factors - allow us to use non-numeric data in statistical modeling
object_columns = all_songs.select_dtypes(include=['object']).columns # First, create list of object columns to convert
all_songs[object_columns] = all_songs[object_columns].astype('category')

In [ ]:
# Making sure there are no null values left in the dataset
nan_rows = all_songs[(all_songs['Genres'].isnull()) | (all_songs['Parent Genres'].isnull())]
nan_rows

In [ ]:
# Splitting 'Parent Genres' column since there are so many different genres
first_instance = all_songs['Parent Genres'].str.split(',').str[0] # extract first genre element

# Assign first instance to new 'Genre' column
all_songs['Genre'] = first_instance

unique_genres = all_songs['Genre'].unique()
num_unique_genres = len(unique_genres)
print("Number of unique genres:", num_unique_genres)

# Counting unique genres
all_songs['Genre'].value_counts() # 17 (now) vs 56 (before)

# Remove unnecessary columns
all_songs = all_songs.drop(columns=['Parent Genres', 'Genres'])

# print(all_songs.dtypes) # Check categorical column types have been converted to string

#### Final Dataset


In [ ]:
# Save dataset as csv file
#all_songs.to_csv('all_spotify_songs.csv')

# Final dataset
all_songs

## Exploratory Data Analysis for Feature Selection

### Correlation Heatmap


In [ ]:
import plotly.graph_objects as go

def corr_plot(data):
    # Calculate the correlation matrix
    correlation_matrix = data.corr()

    # Create heatmap using Plotly  
    annotations = []
    for i, row in enumerate(correlation_matrix.values):
        for j, value in enumerate(row):
            font_color = 'white' if value > -0.4 else '#7fc591'  # Set font color based on z value
            annotations.append(dict(x=correlation_matrix.columns[j], y=correlation_matrix.index[i],
                                text=str(round(value, 2)),
                                showarrow=False, font=dict(color=font_color)))

    # Create heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix.values,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.index,
                    colorscale='Greens',  # Choose your preferred colorscale
                    colorbar=dict(title='Correlation<br>Strength<br>')
    ))



    fig.update_layout(
        title=dict(text ='<b>Correlation Heatmap</b>', x=0.5, y=0.85),
        xaxis=dict(title='<b>Features</b>'),
        yaxis=dict(title='<b>Features</b>'),
        annotations=annotations,
        template="plotly_dark",
        height=500,
        width=700,
        hoverlabel=dict(
            bgcolor="#008000")
    )

    return fig

corr_plot(all_songs)

A correlation heatmap visualizes how well different variables interact with each other.  By illustrating the strength and direction of these relationships, correlation heatmaps help identify patterns, trends, and dependencies within the data. Therefore, we are most interested in the features with very dark or very light tiles.

A few main takeaways:
* **Loud:** Strong positive correlation with Energy suggests that louder songs tend to have higher energy levels.

* **Accoustic:** Strong negative correlation with Energy and Loudness implies that acoustic songs tend to have lower energy and loudness levels.

* **Energy:** Strong positive correlation with Loudness indicates that energetically intense songs tend to be louder.

* **Instrumental:** Moderate negative correlation with Danceability and Popularity suggests that instrumental songs are less danceable and less popular.

* **Dance:** It has a moderate positive correlation with Popularity, Energy, and Happiness, suggesting that more danceable songs tend to be more popular, energetic, and happier.

* **Popularity:** It shows weak positive correlations with attributes like Dance, Energy, Happy, and Loudness, indicating that more popular songs tend to have higher danceability, energy, happiness, and loudness.

### Interactive Scatterplot Comparing Similarity Between Music Tastes


In [ ]:
import plotly.graph_objects as go

# Define colors for each playlist owner
color_map = {'Piero': '#1ED760', 'Nirvit': '#ff00ff'} #1db96e , #b91d82

# Define symbols for each playlist owner
symbol_map = {'Piero': 'circle', 'Nirvit': 'diamond'} #triangle-up

# Define a function to create scatter plot with my original dataset
def create_original_scatter_plot(all_songs):
    # Create scatter plot
    fig = go.Figure()

    # Add text markers when hovering over points
    for group, data in all_songs.groupby('Playlist Owner'):
        fig.add_trace(go.Scatter(
            x=data['Happy'],
            y=data['Energy'],
            opacity=0.75,
            mode='markers',
            name=group,
            text=data.apply(lambda row: f"Song: {row['Song']}, Artist: {row['Artist']}, Energy: {row['Energy']}, Happiness: {row['Happy']}", axis=1),  # Hover text
            marker=dict(
                color=color_map[group],  # Color points based on group
                size=10,
                symbol=symbol_map.get(group, 'circle'),
                line=dict(
                    color='#2a8ccb',
                    width=2
                )
            )
        ))

    # Scatterplot layout
    fig.update_layout(
        title={
            'text': "<b>Top 100 Songs by Mood</b>", # Top 100 Songs by Positivity and Energy Levels
            'font': {'size': 14},
            'x': 0.5,  # Centered title
            'y': 0.9  # Adjust vertical position of title
        },
        xaxis_title="Happiness Level",
        yaxis_title="Energy Level",
        legend_title="Listener",
        width=1070,  # Set width to 1000 pixels
        height=525,  # Set height to 600 pixels
        template="plotly_dark",
        # Make hover text white
        hoverlabel=dict( 
            font=dict(
                color="white"  # Text color inside hover label
            ))
        
    )


    # Label song mood quadrants
    fig.add_annotation(
        x=0, y=105,
        text="<b>Chaotic/Angry</b>",
        font=dict(
            size=12,
            color="white",
        ),
        showarrow=False
    ) 

    fig.add_annotation(
        x= 100, y=105,
        text="<b>Happy/Upbeat</b>",
        font=dict(
            size=12,
            color="white"
        ),
        showarrow=False
    )

    fig.add_annotation(
        x= 100, y=-5,
        text="<b>Chill/Peaceful</b>",
        font=dict(
            size=12,
            color="white"
        ),
        showarrow=False
    )

    fig.add_annotation(
        x=0, y=-5,
        text="<b>Sad/Depressing</b>",
        font=dict(
            size=12,
            color="white"
        ),
        showarrow=False
    )

    # Adding cross section to distinguish mood sectors

    # Vertical line
    fig.add_shape(
        type="line",
        x0=50, y0=0,
        x1=50, y1=100,
        line=dict(
            color="white",
            width=1,
            dash="dash"
        )
    )

    # Horizontal line
    fig.add_shape(
        type="line",
        x0=0, y0=50,
        x1=100, y1=50,
        line=dict(
            color="white",
            width=1,
            dash="dash"
        )
    )

    # Show the plot
    return fig

create_original_scatter_plot(all_songs)

This scatterplot compares the energy and happiness levels of all songs in our Spotify Wrapped playlists. To interpret the plot, it’s important to think about how energy and happiness features interact. 

* Low Energy + Low Happiness =  **Sad / Depressing**

* Low Energy + High Happiness = **Chill / Peaceful**

* High Energy + High Happiness = **Happy / Upbeat**

* High Energy + Low Happiness = **Chaotic / Angry**

The scatterplot reveals that songs from my playlist are primarily clustered in the top quadrant, reflecting a mix of chaotic/angry and happy/upbeat tunes. This clustering pattern could significantly influence a model's predictive capabilities, potentially making the dataset more predictable than anticipated. Additionally, another notable trend emerges: while Nirvit's music taste appears evenly spread across the plot, he tends to gravitate towards a higher proportion of sad and chill music compared to my preferences.

Make sure to hover over the various points on the scatterplot, to see which songs they represent.

## Prepping Data For Machine Learning Models
### Normalize Data


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Remove Artist and Song columns
normalized_songs = all_songs.drop(columns=['Song', 'Artist'])

# Select numerical columns to normalize
columns_to_normalize = ['Popularity', 'BPM', 'Dance', 'Energy', 'Acoustic', 'Instrumental', 'Happy', 'Speech', 'Live', 'Loud', 'Time Signature', 'Time Seconds']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the selected columns
scaler.fit(normalized_songs[columns_to_normalize])

# Transform the selected columns
normalized_songs[columns_to_normalize] = scaler.transform(normalized_songs[columns_to_normalize])

# Create a new binary response column
normalized_songs['Binary Response'] = (normalized_songs['Playlist Owner'] == 'Piero').astype(int)

# Drop the original 'playlist' column if no longer needed
normalized_songs.drop(columns=['Playlist Owner'], inplace=True)

Now we've got ourselves a normalized dataset!


In [ ]:
normalized_songs

### Setting Up Training and Testing Data


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Define features (X) and target variable (y)
X = normalized_songs[['Popularity', 'BPM', 'Dance', 'Energy', 'Acoustic', 'Instrumental', 'Happy', 'Speech', 'Live', 'Loud', 'Key', 'Time Signature', 'Camelot', 'Time Seconds', 'Genre']] # Features

y = normalized_songs['Binary Response'] # Target variable Playlist Owner

# Initialize OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)

# One-hot encode categorical columns
X_encoded = pd.DataFrame(encoder.fit_transform(X[['Key', 'Camelot', 'Genre']]))  # Only encode categorical columns
X_encoded.columns = encoder.get_feature_names_out(['Key', 'Camelot', 'Genre'])  # Get categorical column names

# Reset indices of X and X_encoded
X.reset_index(drop=True, inplace=True)
X_encoded.reset_index(drop=True, inplace=True)

# Concatenate numerical and encoded categorical columns
X_final = pd.concat([X, X_encoded], axis=1)

# Drop original columns since they have been encoded to new columns
X_final.drop(columns=['Key', 'Camelot', 'Genre'], inplace=True)

# Splitting up the data into training and testing sets (60% training, 40% testing)
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.4, random_state=18, shuffle=True)

Now that the data has been split into testing and training sets, the next step involves creating machine learning models to predict which Spotify Wrapped playlist a song belongs to.

## Creating Machine Learning Models

### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create and train the logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy_lr)